In [529]:
import pandas as pd
import numpy as np
import requests
import json
import time
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import uuid
from fake_useragent import UserAgent
import jieba
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, ImageColorGenerator
from os import path
from PIL import Image
from scipy.misc import imread
import re
import nltk

## Gather Data

### Gather Basic Data using Spider

In [530]:
def get_uuid():
    return str(uuid.uuid4())

In [531]:
class LaGouSpider():
    ua = UserAgent()
    cookie = 'JSESSIONID=' + get_uuid() + ';' + \
    'user_trace_token=' + get_uuid() + '; LGUID=' + get_uuid() + '; index_location_city=%E6%88%90%E9%83%BD;' + \
    'SEARCH_ID=' + get_uuid() + '; _gid=GA1.2.717841549.1514043316;' +  \
    '_ga=GA1.2.952298646.1514043316;'\
    'LGSID=' + get_uuid() + ';' +  \
    'LGRID=' + get_uuid() + ';'
    headers = {'cookie': cookie,\
               'user-agent': ua.random,\
               'origin': "https://www.lagou.com",\
               'Host':'www.lagou.com',\
               'x-anit-forge-code': "0",\
               'x-requested-with': "XMLHttpRequest",\
               'x-anit-forge-token': "None",\
               'accept': "application/json, text/javascript, */*; q=0.01",\
               'accept-encoding': "gzip, deflate, br",\
               'accept-language': "zh-CN,zh;q=0.8,en;q=0.6",\
               'content-type': "application/x-www-form-urlencoded; charset=UTF-8",\
               'Referer':'https://www.lagou.com/jobs/list_python?labelWords=&fromSearch=true&suginput=',\
               'cache-control': "no-cache",\
               'postman-token': "91beb456-8dd9-0390-a3a5-64ff3936fa63"}
    def __init__(self, city = '北京', keyword = '数据分析', page_num = 1):
        self.para = {  
            'first': 'true',  
            'pn': '1',  
            'kd': keyword,  
            'city': city
            }
        self.page_num = page_num
        self.proxies = {'http':'http://119.28.142.148:8888',\
            'https':'https://119.28.142.148:8888'}
    def spider(self):
        position_list = []
        try: 
            for pn_loop in range(0, self.page_num+1):
                print('{:d} begins'.format(pn_loop) )
                if pn_loop == 1:
                    pass
                else:
                    self.para['first'] = 'False'
                    self.para['pn'] = str(pn_loop)
                response = requests.get('https://www.lagou.com/jobs/positionAjax.json?needAddtionalResult=false&isSchoolJob=0',
                       headers=headers, params=self.para, proxies = self.proxies)
                response_json = response.json()
                print(response_json)
                position_inf = response_json['content']['positionResult']['result']
                position_list.extend(position_inf)
                time.sleep(np.random.randint(5,10))
        except:
            print('Someting wrong here')
        return position_list

In [532]:
#fenxi_spider = LaGouSpider(city = '北京', keyword = '数据分析', page_num = 30)

In [533]:
#position_list = fenxi_spider.spider()

In [534]:
#len(position_list)

In [535]:
#wajue_spider = LaGouSpider(city = '北京', keyword = '数据挖掘', page_num = 30)

In [536]:
#position_list_wajue = wajue_spider.spider()

In [537]:
#len(position_list_wajue)

### Save part of Data to DataFrame using Pandas

In [538]:
class SelectData():
    def __init__(self, position_list):
        self.position_list = position_list
    def select_data(self):
        data = pd.DataFrame()
        for loop in range(len(self.position_list)):
            data.loc[loop, 'city'] = self.position_list[loop]['city']
            data.loc[loop, 'companyFullName'] = self.position_list[loop]['companyFullName']
            data.loc[loop, 'companyShortName'] = self.position_list[loop]['companyShortName']
            data.loc[loop, 'companySize'] = self.position_list[loop]['companySize']
            data.loc[loop, 'district'] = self.position_list[loop]['district']
            data.loc[loop, 'education'] = self.position_list[loop]['education']
            data.loc[loop, 'financeStage'] = self.position_list[loop]['financeStage']
            data.loc[loop, 'positionName'] = self.position_list[loop]['positionName']
            data.loc[loop, 'salary'] = self.position_list[loop]['salary']
            data.loc[loop, 'workYear'] = self.position_list[loop]['workYear']
            data.loc[loop, 'positionId'] = self.position_list[loop]['positionId']
        return data

In [539]:
#data_fenxi = SelectData(position_list)
#data_fenxi = data_fenxi.select_data()
#data_fenxi['positionId'] = data_fenxi.positionId.astype(int)

In [540]:
#data_fenxi

In [541]:
#data_fenxi.to_csv('shujufenxi_basic.csv', encoding="utf_8_sig", index=False)

In [542]:
#list(data_fenxi.positionId)

In [543]:
#data_wajue = SelectData(position_list_wajue)
#data_wajue = data_wajue.select_data()
#data_wajue['positionId'] = data_wajue.positionId.astype(int)

In [544]:
#data_wajue.to_csv('shujuwajue_basic.csv', encoding="utf_8_sig", index=False)

In [545]:
#data_wajue.positionId.drop_duplicates

### Gather more Detailed Data based on PositionId

In [546]:
class CrawlDetail():
    headers = {  
        'Host':'www.lagou.com',  
        'Upgrade-Insecure-Requests':'1',  
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.119 Safari/537.36'  
    }  
    def __init__(self, positionId_list):
        self.positionId_list = positionId_list
    def crawl(self):
        data_detail = pd.DataFrame()
        max_retry = 8
        for index, positionId in enumerate(self.positionId_list):
            print(positionId)
            # the first try
            url = 'https://www.lagou.com/jobs/%s.html' %positionId
            response = requests.get(url, headers = headers)
            soup = BeautifulSoup(response.content, 'html.parser') 
            if soup.find('dd', attrs={'class':'job_bt'}) == None: # the first try fail
                print ('the first try fail')
                none_flag = True
                retry_num = 0
                data_detail.loc[index, 'positionId'] = self.positionId_list[index]
                while ((none_flag == True) & (retry_num < max_retry)):
                    #print ('retry again')
                    retry_num += 1
                    response = requests.get(url, headers = headers)
                    soup = BeautifulSoup(response.content, 'html.parser')
                    if soup.find('dd', attrs={'class':'job_bt'}) == None:
                        none_flag = True
                        time.sleep(5)
                    else:
                        print('retry {:d} times, {:d}'.format(retry_num, positionId))
                        none_flag = False 
                        data_detail.loc[index, 'detail'] = soup.find('dd', attrs={'class':'job_bt'}).text    
                if retry_num == max_retry:
                    data_detail.loc[index, 'detail'] = None
            else: # the first try succeed 
                print ('the first try succeed')
                data_detail.loc[index, 'positionId'] = self.positionId_list[index]
                data_detail.loc[index, 'detail'] = soup.find('dd', attrs={'class':'job_bt'}).text
            sleep_random = np.random.randint(5,10)
            time.sleep(sleep_random)
        return data_detail     

In [547]:
#fenxi_crawl_detail = CrawlDetail(list(data_fenxi.positionId))

In [548]:
#data_fenxi_detail = fenxi_crawl_detail.crawl()

In [549]:
#len(data_fenxi_detail)

In [550]:
#wajue_crawl_detail = CrawlDetail(list(data_wajue.positionId))

In [551]:
#data_wajue_detail = wajue_crawl_detail.crawl()

In [552]:
#len(data_wajue_detail)

In [553]:
#data_fenxi_detail.to_csv('shujufenxi_detail.csv', encoding="utf_8_sig", index=False)

In [903]:
#data_wajue_detail.to_csv('shujuwajue_detail.csv', encoding="utf_8_sig", index=False)

## Read Data in

In [590]:
data_fenxi = pd.read_csv('shujufenxi_basic.csv')
data_wajue = pd.read_csv('shujuwajue_basic.csv')

In [591]:
data_fenxi_detail = pd.read_csv('shujufenxi_detail.csv')
data_wajue_detail = pd.read_csv('shujuwajue_detail.csv')

## Assess 
- merge detail inf
- add one column to both fenxi and wajue to seperate the '数据分析' and ‘数据入门’
- since re-crawl part of wajue data, drop the duplicated ones
- merge fenxi and wajue data together
- reset index for the data_all
- save data_all to csv file

## Clean 
- copy data_all for cleanning
- drop positonName column since we have kd column
- delete the '\n' in detail column
- (ing) split the detail column to seperate part, e.g., 工作职责，任职要求
- not good the split part, keep the sub data to dig currently, if there's time later, revise
- add salary_avg column 

- merge detail inf and add one column to both fenxi and wajue to seperate the '数据分析' and ‘数据入门’

In [604]:
data_fenxi_all = pd.merge(data_fenxi, data_fenxi_detail, on='positionId')
data_fenxi_all['kd'] = 'fenxi'
len(data_fenxi_all)
data_fenxi_all.head()

In [598]:
#data_fenxi_all = data_fenxi_all.drop(['Unnamed: 0'],axis=1)

In [605]:
data_wajue_all = pd.merge(data_wajue, data_wajue_detail, on='positionId')
len(data_wajue_all)
data_wajue_all['kd'] = 'wajue'
data_wajue_all.head()

In [601]:
#data_wajue_all = data_wajue_all.drop(['Unnamed: 0'],axis=1)

- re-crawl part of wajue data, drop the duplicated ones

In [602]:
data_wajue_all = data_wajue_all.drop_duplicates()

In [603]:
len(data_wajue_all)

445

- merge fenxi and wajue data together

In [606]:
data_all = pd.concat([data_fenxi_all, data_wajue_all])

In [607]:
len(data_all)

895

In [608]:
data_all.head()

,city,companyFullName,companyShortName,companySize,district,education,financeStage,positionName,salary,workYear,positionId,detail,kd
0,北京,汇联金科信息技术（北京 )有限公司,汇联金科,50-150人,朝阳区,本科,A轮,数据分析师,10k-15k,3-5年,4295741,\n职位描述：\n\n工作职责：1. 根据业务实际需求，制定相应策略，完成深入的运营数据分析...,fenxi
1,北京,途家网网络技术（北京）有限公司,途家网,500-2000人,朝阳区,本科,D轮及以上,数据分析师,15k-30k,3-5年,4505031,\n职位描述：\n\n工作职责：1、通过SQL收集数据，分析流量、活动、订单等核心数据，形成...,fenxi
2,北京,和信电子商务有限公司,和信贷,150-500人,朝阳区,大专,上市公司,数据分析师,6k-8k,3-5年,4507358,\n职位描述：\n\n\n岗位职责：\n1. 对数据敏感，善于发现业务中存在的问题，并给出优...,fenxi
3,北京,京东金融,京东金融,2000人以上,大兴区,本科,上市公司,数据分析师,14k-27k,3-5年,4505903,\n职位描述：\n\n职责描述：\n1、负责定期编制金融集团合并报表，完成报表变动分析； ...,fenxi
4,北京,杭州云脑科技有限公司,云脑科技,15-50人,朝阳区,本科,A轮,数据分析师,15k-30k,3-5年,3898857,\n职位描述：\n\n【岗位职责】\n1、收集和整理公司运营数据、竞品数据、行业数据，解读趋...,fenxi


- reset index for the data_all

In [609]:
data_all.reset_index(inplace=True, drop=True)

- save data_all

In [610]:
data_all.to_csv('data_all.csv', encoding="utf_8_sig", index=False)

- read data_all

In [611]:
data_all = pd.read_csv('data_all.csv')

In [612]:
data_all.head()

,city,companyFullName,companyShortName,companySize,district,education,financeStage,positionName,salary,workYear,positionId,detail,kd
0,北京,汇联金科信息技术（北京 )有限公司,汇联金科,50-150人,朝阳区,本科,A轮,数据分析师,10k-15k,3-5年,4295741,\n职位描述：\n\n工作职责：1. 根据业务实际需求，制定相应策略，完成深入的运营数据分析...,fenxi
1,北京,途家网网络技术（北京）有限公司,途家网,500-2000人,朝阳区,本科,D轮及以上,数据分析师,15k-30k,3-5年,4505031,\n职位描述：\n\n工作职责：1、通过SQL收集数据，分析流量、活动、订单等核心数据，形成...,fenxi
2,北京,和信电子商务有限公司,和信贷,150-500人,朝阳区,大专,上市公司,数据分析师,6k-8k,3-5年,4507358,\n职位描述：\n\n\n岗位职责：\n1. 对数据敏感，善于发现业务中存在的问题，并给出优...,fenxi
3,北京,京东金融,京东金融,2000人以上,大兴区,本科,上市公司,数据分析师,14k-27k,3-5年,4505903,\n职位描述：\n\n职责描述：\n1、负责定期编制金融集团合并报表，完成报表变动分析； ...,fenxi
4,北京,杭州云脑科技有限公司,云脑科技,15-50人,朝阳区,本科,A轮,数据分析师,15k-30k,3-5年,3898857,\n职位描述：\n\n【岗位职责】\n1、收集和整理公司运营数据、竞品数据、行业数据，解读趋...,fenxi


- copy data for cleanning

In [613]:
data_all_clean = data_all.copy()

- drop positonName column since we have kd column

In [614]:
data_all_clean.drop('positionName', axis=1, inplace=True)

- delete the '\n' in detail column

In [615]:
data_all_clean['detail'] = data_all_clean['detail'].str.replace('\n', '')

In [616]:
data_all_clean.sample(5)

,city,companyFullName,companyShortName,companySize,district,education,financeStage,salary,workYear,positionId,detail,kd
535,北京,完美世界（北京）软件科技发展有限公司,完美世界,2000人以上,朝阳区,本科,上市公司,18k-30k,1-3年,4323681,职位描述：工作职责：1) 参与数据清洗，大数据处理，数据挖掘相关工作 2) 参与各种核心搜索...,wajue
34,北京,飞驰镁物（北京）信息服务有限公司,飞驰镁物,150-500人,朝阳区,本科,B轮,13k-20k,3-5年,3466215,职位描述：岗位职责：1. 负责数据挖掘分析需求的沟通、收集、整理、理解和实现；2. 负责产品...,fenxi
375,北京,北京慢点生活科技有限公司,动动,15-50人,朝阳区,本科,B轮,15k-25k,不限,1986051,职位描述：【我们的动动情怀】我们是慢点科技荣誉出品的动动/Pacer，我们致力于用技术的手段...,fenxi
81,北京,北京国双科技有限公司,Gridsum 国双,500-2000人,海淀区,本科,上市公司,15k-30k,3-5年,920876,职位描述：在国双数据中心，你可以领略国内外数据行业激动人心的商业应用，拥有与全球高端客户并肩...,fenxi
6,北京,随行付支付有限公司,随行付支付有限公司,500-2000人,石景山区,本科,上市公司,20k-30k,3-5年,4042512,职位描述：岗位职责：1、支付业务的相关数据梳理；2、对业务各环节产生的数据，做统计与深度运营...,fenxi


In [617]:
data_all_clean['detail'][0]

'职位描述：工作职责：1. 根据业务实际需求，制定相应策略，完成深入的运营数据分析报告，对用户活跃、留存、有效行为等指标负责；2. 对运营数据进行分析与监控，能够基于数据进行用户画像描绘、用户分层，并针对不同用户指定不同的运营策略，拉新、促活；3. 统计分析相关业务数据，敏锐地发现异常，针对风险点进行持续追踪与预警；4、结合行业数据、业界观点和市场趋势，为长期业务发展方向提供战略层面的意见和建议任职要求：1. 本科及以上学历，经济、统计、数学、计算机等相关专业背景优先；2. 至少3年以上互联网金融行业用户研究、数据分析、数据运营、数据挖掘等相关经验；3 具备优秀的数据分析能力及策略制定能力，能够快速理解业务，有独立思考能力并提出自己的见解；4. 对数据敏感，能从海量数据提炼核心结果，具备优秀的数据处理能力、报告呈现于解说能力；5. 沟通能力强，具备一定的抗压性6.\xa0熟练使用Excel，包括图表制作，函数及数据透视表运用等。熟练使用数理统计、数据分析、数据挖掘工具软件（如SPSS，SAS,Clementine 等）。精通数据分析方法：回归，时间序列，神经网络，聚类分析，关联分析，数据建模等分析方法'

- split the detail column to seperate part, i.e., 工作职责，任职要求

In [618]:
list1 = data_all_clean[pd.isna(data_all_clean['detail'].str.extract('(1.*)1.*'))].index

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [619]:
list2 = data_all_clean[pd.isna(data_all_clean['detail'].str.extract('1.*(1.*)'))].index

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [620]:
set(list1) == set(list2)

True

In [621]:
data_all_clean['task'] = data_all_clean['detail'].str.extract('(1.*)1.*')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [622]:
data_all_clean['skill'] = data_all_clean['detail'].str.extract('1.*(1.*)')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [623]:
data_all_clean.head()

,city,companyFullName,companyShortName,companySize,district,education,financeStage,salary,workYear,positionId,detail,kd,task,skill
0,北京,汇联金科信息技术（北京 )有限公司,汇联金科,50-150人,朝阳区,本科,A轮,10k-15k,3-5年,4295741,职位描述：工作职责：1. 根据业务实际需求，制定相应策略，完成深入的运营数据分析报告，对用户...,fenxi,1. 根据业务实际需求，制定相应策略，完成深入的运营数据分析报告，对用户活跃、留存、有效行为...,1. 本科及以上学历，经济、统计、数学、计算机等相关专业背景优先；2. 至少3年以上互联网金...
1,北京,途家网网络技术（北京）有限公司,途家网,500-2000人,朝阳区,本科,D轮及以上,15k-30k,3-5年,4505031,职位描述：工作职责：1、通过SQL收集数据，分析流量、活动、订单等核心数据，形成可视化报表；...,fenxi,1、通过SQL收集数据，分析流量、活动、订单等核心数据，形成可视化报表；2、分析用户行为、特...,1、3年以上互联网数据经验2、逻辑能力强，擅长数据分析与需求调研3、有很强的数据调研分析能力...
2,北京,和信电子商务有限公司,和信贷,150-500人,朝阳区,大专,上市公司,6k-8k,3-5年,4507358,职位描述：岗位职责：1. 对数据敏感，善于发现业务中存在的问题，并给出优化建议；2.熟悉用户...,fenxi,1. 对数据敏感，善于发现业务中存在的问题，并给出优化建议；2.熟悉用户生命周期管理建设，搭...,1、 金融、统计、数学、计算机等专业优先；2、 熟练使用Excel、SQL、SPSS，能够使...
3,北京,京东金融,京东金融,2000人以上,大兴区,本科,上市公司,14k-27k,3-5年,4505903,职位描述：职责描述：1、负责定期编制金融集团合并报表，完成报表变动分析； ...,fenxi,1、负责定期编制金融集团合并报表，完成报表变动分析； ...,1、计算机、统计学、数学、数理统计等相关专业，本科以上学历，3年以上相关工作经验； 2、熟练...
4,北京,杭州云脑科技有限公司,云脑科技,15-50人,朝阳区,本科,A轮,15k-30k,3-5年,3898857,职位描述：【岗位职责】1、收集和整理公司运营数据、竞品数据、行业数据，解读趋势变化和异常波动...,fenxi,1、收集和整理公司运营数据、竞品数据、行业数据，解读趋势变化和异常波动，撰写分析报告；2、针...,1、计算机、数学等理工科专业本科以上学历；2、思路清晰，逻辑性强，具有较强的业务理解能力和表...


- not good the split part, keep the sub data to dig currently, if there's time later, revise

In [624]:
data_all_clean_sub = data_all_clean[~pd.isna(data_all_clean['task'])]

In [625]:
data_all_clean_sub.sample(2)

,city,companyFullName,companyShortName,companySize,district,education,financeStage,salary,workYear,positionId,detail,kd,task,skill
388,北京,东峡大通（北京）管理咨询有限公司,ofo,2000人以上,海淀区,本科,D轮及以上,17k-34k,3-5年,4444515,职位描述：工作职责:1. 监测重要业务指标数据变化，分析变化趋势和原因，并形成周期性报告2....,fenxi,1. 监测重要业务指标数据变化，分析变化趋势和原因，并形成周期性报告2. 建立业务健康度模型...,1硕士学位；数学，应用经济学，统计学等相关专业优先
326,北京,达飞云贷科技（北京）有限公司,达飞云贷,2000人以上,朝阳区,本科,不需要融资,15k-25k,3-5年,3754688,职位描述：岗位职责：1、调研、分析业务部门数据需求；2、发现运营问题，进行深度专题分析，形成...,fenxi,1、调研、分析业务部门数据需求；2、发现运营问题，进行深度专题分析，形成结论并独立撰写分析报...,1、本科学历，统计学、计量经济学或者数学专业优先；2、精通Excel、熟练使用R、pytho...


In [626]:
len(data_all_clean_sub[data_all_clean_sub['kd'] == 'fenxi'])

416

In [627]:
len(data_all_clean_sub[data_all_clean_sub['kd'] == 'wajue'])

363

- add salary_avg column 

In [644]:
data_all_clean_sub['salary_low'] = data_all_clean_sub.salary.str.extract('(.*).-')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [645]:
#salary_low.sort_values()

In [646]:
data_all_clean_sub['salary_up'] = data_all_clean_sub.salary.str.extract('-(.*).')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [647]:
data_all_clean_sub['salary_low'] = data_all_clean_sub['salary_low'].astype(float)
data_all_clean_sub['salary_up'] = data_all_clean_sub['salary_up'].astype(float)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [648]:
data_all_clean_sub['salary_avg'] = (data_all_clean_sub['salary_low'] + 
                                    data_all_clean_sub['salary_up'])/2

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [649]:
data_all_clean_sub.to_csv('data_all_clean_sub.csv', encoding="utf_8_sig", index=False)

In [676]:
data_all_clean_sub.groupby(['kd','workYear']).salary_up.max()

kd     workYear
fenxi  1-3年         40.0
       1年以下          7.0
       3-5年         60.0
       5-10年       100.0
       不限           40.0
wajue  1-3年         50.0
       1年以下         20.0
       3-5年        100.0
       5-10年        80.0
       不限           60.0
Name: salary_up, dtype: float64

In [677]:
data_all_clean_sub.groupby(['kd','workYear']).salary_up.min()

kd     workYear
fenxi  1-3年         6.0
       1年以下         7.0
       3-5年         8.0
       5-10年       15.0
       不限           8.0
wajue  1-3年        12.0
       1年以下        19.0
       3-5年         5.0
       5-10年       25.0
       不限           6.0
Name: salary_up, dtype: float64

In [683]:
data_all_clean_sub[(data_all_clean_sub['salary_low'] == 5) & (data_all_clean_sub['kd'] == 'wajue')]

,city,companyFullName,companyShortName,companySize,district,education,financeStage,salary,workYear,positionId,detail,kd,task,skill,salary_low,salary_up,salary_avg
572,北京,北京思维造物信息科技有限公司,罗辑思维,150-500人,朝阳区,本科,B轮,5k-10k,不限,4501239,职位描述：职责描述：1.负责大数据平台的建设和数据开发工作2.负责使用数据分析、挖掘相关算法...,wajue,1.负责大数据平台的建设和数据开发工作2.负责使用数据分析、挖掘相关算法和工具进行产品开发和...,1.计算机相关专业本科及以上学历，有扎实的机器学习/数据挖掘/自然语言处理基础2.每周保证4...,5.0,10.0,7.5


In [651]:
#data_all_clean_sub['salary_avg']

## Analysis 
- will use tableau to do basic comparing
- use ntlk, wordcloud to do basic wordcloud

- use ntlk, wordcloud to do basic wordcloud

In [652]:
skill_fenxi = data_all_clean_sub[data_all_clean_sub['kd'] == 'fenxi']['skill'].str.cat(sep = ' ')
task_fenxi = data_all_clean_sub[data_all_clean_sub['kd'] == 'fenxi']['task'].str.cat(sep = ' ')

In [653]:
len(skill_fenxi)

76442

In [654]:
skill_wajue = data_all_clean_sub[data_all_clean_sub['kd'] == 'wajue']['skill'].str.cat(sep = ' ')
task_wajue = data_all_clean_sub[data_all_clean_sub['kd'] == 'wajue']['task'].str.cat(sep = ' ')

In [655]:
len(skill_wajue)

71749

In [656]:
skill_fenxi_cut = jieba.cut(skill_fenxi)
skill_wajue_cut = jieba.cut(skill_wajue)
task_fenxi_cut = jieba.cut(task_fenxi)
task_wajue_cut = jieba.cut(task_wajue)

In [657]:
skill_fenxi_cut_text = ",".join(skill_fenxi_cut)
skill_wajue_cut_text = ",".join(skill_wajue_cut)
task_fenxi_cut_text = ",".join(task_fenxi_cut)
task_wajue_cut_text = ",".join(task_wajue_cut)

In [658]:
print("/".join(skill_fenxi_cut))

In [659]:
stopwords = pd.read_table('stopwords.dat')
stopwords.columns = ['stopwords']
stopwords = stopwords.stopwords.str.cat(sep = ',')
stopwords = stopwords + ',1,2,3,4,5,6,7,8,9,10,，,、, ,；,\xa0,。,：,）,（,:,+,】,【,;,—,“,”,「,」,-,．,&,！'

In [660]:
def filter_stopwords(stopwords, text):
    text_list = text.split(',')
    filter_list = [word for word in text_list if word not in stopwords]
    filter_text = ','.join(filter_list)
    return filter_text 

In [661]:
def make_freq_list(text):
    stopwords_mini = 'A, APP, L, O, P, PIG, COM, HTTP, HTTPS, ISHUMEI, WWW'
    out = re.findall('([a-zA-Z]*)', text)
    out = [loop.upper() for loop in out if loop.upper() not in stopwords_mini]
    out = ','.join(out)
    out = out.split(',')
    out_freq = nltk.FreqDist(out)
    for loop in list(out_freq.keys()):
        if out_freq[loop] == 1:
            del(out_freq[loop])
    return out_freq

In [662]:
class MakeWordCloud():
    def __init__(self, text, mask = None, font = 'HYQiHei-25J.ttf', out_name = 'skill_fenxi.jpg'):
        self.text = text
        self.mask = mask
        self.font = font
        self.out_name = out_name
    def make_cloud(self, max_words = 1000):
        cur_path = path.abspath('.')
        if self.mask == None:
            color_mask = None
        else:
            color_mask = imread(self.mask)
        wc = WordCloud(font_path=self.font, background_color="white", 
               max_words=max_words, mask=color_mask, random_state=42, max_font_size=100,width=500, height=350)
        cloud = wc.generate_from_frequencies(self.text)
        if self.mask == None:
            pass
        else:
            img_colors = ImageColorGenerator(color_mask) 
            cloud.recolor(color_func = img_colors)
        cloud.to_file(path.join(cur_path, self.out_name))

In [663]:
filter_text = filter_stopwords(stopwords, skill_fenxi_cut_text)
skill_fenxi_freq = make_freq_list(filter_text)
skill_fenxi_cloud = MakeWordCloud(skill_fenxi_freq, out_name='skill_fenxi.jpg')
skill_fenxi_cloud.make_cloud()

In [664]:
filter_text = filter_stopwords(stopwords, skill_wajue_cut_text)
skill_wajue_freq = make_freq_list(filter_text)
skill_wajue_cloud = MakeWordCloud(skill_wajue_freq, out_name='skill_wajue.jpg')
skill_wajue_cloud.make_cloud()

In [665]:
filter_text = filter_stopwords(stopwords, task_fenxi_cut_text)
task_fenxi_freq = nltk.FreqDist(filter_text.split(','))
task_fenxi_cloud = MakeWordCloud(task_fenxi_freq, mask = 'zhichangren.jpg', out_name='task_fenxi.jpg')
task_fenxi_cloud.make_cloud()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if sys.path[0] == '':


In [666]:
filter_text = filter_stopwords(stopwords, task_wajue_cut_text)
task_wajue_freq = nltk.FreqDist(filter_text.split(','))
task_wajue_cloud = MakeWordCloud(task_wajue_freq, mask = 'zhichangren.jpg', out_name='task_wajue.jpg')
task_wajue_cloud.make_cloud()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if sys.path[0] == '':
